In [ ]:
# rxperiment will run faster on gpu, select GPU runtime
!pip install gdown
zip_file_id="1ExZ-N4_RcMzNZO2uI-wo7pJqAFY135X9"
!gdown --id $zip_file_id
!mkdir -p chest_xray && unzip -o chest_xray.zip 
!rm chest_xray.zip

In [2]:
import os
try:
    import numpy as np
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    os.system("pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113")
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    os.system("pip install -q torchinfo")
    from torchinfo import summary

from pathlib import Path

import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
NUM_WORKERS = os.cpu_count()

torch version: 1.13.1+cu116
torchvision version: 0.14.1+cu116
[INFO] Couldn't find torchinfo... installing it.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# training and testing block

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
 
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:


    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    

    model.to(device)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


In [5]:
image_path = "/content/chest_xray" 



# Setup Dirs
train_dir = image_path + "/train"
test_dir = image_path + "/test"


manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])


In [6]:


def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
  

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)


  class_names = train_data.classes


  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names


train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms, # resize, convert images to between 0 & 1 and normalize them
                                                                               batch_size=32) # set mini-batch size to 32


We will have 3 experiments one for resnet18, efficientnet-b0 and densenet121 each. 

For resnet18

In [7]:
weights = torchvision.models.ResNet18_Weights.DEFAULT 
model = torchvision.models.resnet18(pretrained=True).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True 


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=512, 
                    out_features=output_shape, 
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best result for resnet18 was 91.25%

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.2765 | train_acc: 0.9454 | test_loss: 0.6914 | test_acc: 0.8375
Epoch: 2 | train_loss: 0.0501 | train_acc: 0.9797 | test_loss: 0.7724 | test_acc: 0.8562
Epoch: 3 | train_loss: 0.0309 | train_acc: 0.9872 | test_loss: 0.4276 | test_acc: 0.9125
Epoch: 4 | train_loss: 0.0456 | train_acc: 0.9837 | test_loss: 1.9729 | test_acc: 0.7109
Epoch: 5 | train_loss: 0.0183 | train_acc: 0.9946 | test_loss: 2.1544 | test_acc: 0.7250
Epoch: 6 | train_loss: 0.0185 | train_acc: 0.9925 | test_loss: 2.1859 | test_acc: 0.7188
Epoch: 7 | train_loss: 0.0317 | train_acc: 0.9875 | test_loss: 0.9228 | test_acc: 0.8047
Epoch: 8 | train_loss: 0.0237 | train_acc: 0.9921 | test_loss: 1.3612 | test_acc: 0.8187
Epoch: 9 | train_loss: 0.0171 | train_acc: 0.9935 | test_loss: 1.3736 | test_acc: 0.7906
Epoch: 10 | train_loss: 0.0176 | train_acc: 0.9933 | test_loss: 0.9375 | test_acc: 0.8391


For efficientnet-b0

In [9]:
weight = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights
model = torchvision.models.efficientnet_b0(weights=weight).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True 


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape, 
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best 81.87%

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1182 | train_acc: 0.9561 | test_loss: 1.1258 | test_acc: 0.6922
Epoch: 2 | train_loss: 0.0552 | train_acc: 0.9780 | test_loss: 0.7274 | test_acc: 0.8187
Epoch: 3 | train_loss: 0.0394 | train_acc: 0.9868 | test_loss: 1.0870 | test_acc: 0.7562
Epoch: 4 | train_loss: 0.0208 | train_acc: 0.9929 | test_loss: 1.0379 | test_acc: 0.7766
Epoch: 5 | train_loss: 0.0111 | train_acc: 0.9965 | test_loss: 1.2499 | test_acc: 0.7594
Epoch: 6 | train_loss: 0.0115 | train_acc: 0.9960 | test_loss: 1.0547 | test_acc: 0.7781
Epoch: 7 | train_loss: 0.0252 | train_acc: 0.9914 | test_loss: 0.9277 | test_acc: 0.7609
Epoch: 8 | train_loss: 0.0090 | train_acc: 0.9971 | test_loss: 0.9773 | test_acc: 0.7672
Epoch: 9 | train_loss: 0.0046 | train_acc: 0.9985 | test_loss: 1.2466 | test_acc: 0.7734
Epoch: 10 | train_loss: 0.0135 | train_acc: 0.9964 | test_loss: 1.5296 | test_acc: 0.7125


For densenet121 

In [10]:
weights = torchvision.models.DenseNet121_Weights.DEFAULT
model = torchvision.models.densenet121(weights=weights).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train all params
for param in model.parameters():
    param.requires_grad = True 


torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)


model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1024, 
                    out_features=output_shape, 
                    bias=True)).to(device)


results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device) # best is 92.81

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1252 | train_acc: 0.9538 | test_loss: 0.4706 | test_acc: 0.8391
Epoch: 2 | train_loss: 0.0736 | train_acc: 0.9730 | test_loss: 0.1970 | test_acc: 0.9281
Epoch: 3 | train_loss: 0.0622 | train_acc: 0.9785 | test_loss: 0.4847 | test_acc: 0.8656
Epoch: 4 | train_loss: 0.0462 | train_acc: 0.9841 | test_loss: 0.6712 | test_acc: 0.8172
Epoch: 5 | train_loss: 0.0344 | train_acc: 0.9885 | test_loss: 0.7517 | test_acc: 0.7969
Epoch: 6 | train_loss: 0.0404 | train_acc: 0.9856 | test_loss: 0.8014 | test_acc: 0.7875
Epoch: 7 | train_loss: 0.0328 | train_acc: 0.9868 | test_loss: 0.7087 | test_acc: 0.8281
Epoch: 8 | train_loss: 0.0251 | train_acc: 0.9904 | test_loss: 0.3819 | test_acc: 0.8594
Epoch: 9 | train_loss: 0.0226 | train_acc: 0.9902 | test_loss: 0.8854 | test_acc: 0.7984
Epoch: 10 | train_loss: 0.0186 | train_acc: 0.9942 | test_loss: 0.5339 | test_acc: 0.8547
